In [1]:
import time
from datetime import datetime
import numpy as np
import pandas as pd
from scipy.optimize import brute
import configparser
from glob import glob

pd.set_option('precision', 9)

from talib import RSI, ATR

#from data_read import check_time_range

In [6]:
def check_time_range(df) :
    t1 = df.iloc[0]['T']/1000
    t2 = df.iloc[-1]['T']/1000

    return t2-t1

#df_trades = pd.DataFrame(columns = ['Date', 'Trade type', 'Price', 'cumProfit', 'curCapital', 'rsiVal' ])   
def RSI03(params, df):

    rsiWin =  params[0]     
    rsiBuy = params[1] 
    rsiSell = params[2] 
    numAtrLoss = params[3]
    
    data_len = df.shape[0]
    myDate  = df["T"].tolist()
    myHigh  = df["H"].as_matrix()
    myLow   = df["L"].as_matrix()
    myClose = df["C"].as_matrix()
    
    rsiArray = RSI(myClose * 100, rsiWin) 
    atrArray = ATR(myHigh, myLow, myClose, 10)
  
    marketPosition = 0
    barsSinceEntry = 0    
    startCapital = 0.1
    curCapital = startCapital
    cumProfit = 0
    buy_sum = 0
    num_trades = 0
    deposit = 0
    
    startDate = rsiWin
    for i in range(startDate, data_len):
     
   
            rsiVal = rsiArray[i]
            atrVal = atrArray[i]
            
            #Long entry 
            if marketPosition == 0 and rsiVal <= rsiBuy:
                
                price = myClose[i] * buyCorrection
                
                numShares = curCapital / price * (1-commision)       
                buy_sum = curCapital 
                curCapital = 0
                
                marketPosition = 1
                entryPrice = price            
                #df_trades.loc[len(df_trades)] = [myDate[i], "Long entry", price, cumProfit, curCapital, rsiVal]
                num_trades += 1

                barsSinceEntry = 0            
            #Long Exit Profit 
            if marketPosition == 1 and rsiVal > rsiSell and barsSinceEntry > 1:
                price = myClose[i] * sellCorrection
                
                profit = price * numShares * (1-commision) 
     
                pure_profit = profit - buy_sum
                cumProfit += pure_profit
                deposit_sum = max(0, pure_profit) * deposit_percent
                deposit += deposit_sum
                curCapital = profit - deposit_sum
     
             
                marketPosition = 0 
                #df_trades.loc[len(df_trades)] = [myDate[i], "Long profit", price,  cumProfit, curCapital, rsiVal]
                
            #Long Exit Loss
            if marketPosition == 1 and myClose[i] < entryPrice - numAtrLoss * atrVal and barsSinceEntry > 1:
                price = myClose[i] * sellCorrection
                                    
                profit = price * numShares * (1-commision)
                cumProfit += profit - buy_sum
                curCapital = profit 
                       
                marketPosition = 0
                #df_trades.loc[len(df_trades)] = [myDate[i], "Long loss", price, cumProfit, curCapital, rsiVal]
            barsSinceEntry += 1         
            
    return (cumProfit, num_trades, deposit)

commision = 0.0015

#oneMin, fiveMin, thirtyMin, hour
config = configparser.ConfigParser()
config.read("config.ini")
data_path = config["DEFAULT"]['data_path']
data_path = data_path + '/TRX-BTC/3m/'

buyCorrection = 1.004
sellCorrection = 0.996
deposit_percent = 0

fn = glob(data_path+'*csv')[-1]
df = pd.read_csv(fn);
print("Rows: ", len(df))

t1 = df.iloc[0]['T']/1000
t1 = datetime.utcfromtimestamp(t1).strftime('%Y-%m-%dT%H:%M:%SZ')
print("Start time:", t1)
t2 = df.iloc[-1]['T']/1000
t2 = datetime.utcfromtimestamp(t2).strftime('%Y-%m-%dT%H:%M:%SZ')
print("End time:", t2)
tr = check_time_range(df)
tr = round(tr/3600/24)
print("Дней:", tr)

Rows:  39554
Start time: 2018-01-01T00:00:00Z
End time: 2018-03-25T06:39:00Z
Дней: 83.0


In [5]:
data_path

'G:/Code/Projects/Quant Mod/Robots/Data/TRX-BTC/3m'

In [18]:
res = []
def fo(x):

    profit, num_trades, deposit = RSI03(x, df) 
    
    res.append([profit, num_trades, deposit] + x.tolist() )
    return profit

t1 = time.time()
brute(fo, ((5,20,1), (30,48,2), (50,80,2), (6,11,1)), finish=None)
#brute(fo, ((10,17,1), (28,46,1), (70,80,1), (6,11,1)), finish=None)

t2 = time.time()
print(round(t2-t1), ' sec')

531  sec


In [19]:
res = np.asarray(res)
dres = pd.DataFrame(res) 

dres['PPD'] = dres[0]/tr * 1000
dres['PNT'] = dres[0] * dres[1]
dres1 = dres[(dres['PPD'] > 2) & (dres[1] > 0) ]
dres1 = dres1.sort_values('PNT', ascending=False)
print(data_path)
print("Дней:", tr)
#print(dres1)
pd.set_option('display.precision', 4)
#with pd.option_context('display.max_columns', 12):
print(dres1)

Data/NCASH-BTC/3m/
Дней: 26.0
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, PPD, PNT]
Index: []


In [20]:
dres[dres[0]>0]

,0,1,2,3,4,5,6,PPD,PNT


In [27]:
res = np.asarray(res)
dres = pd.DataFrame(res) 

dres['PPD'] = dres[0]/tr * 1000
dres['PNT'] = dres[0] * dres[1]
dres1 = dres[(dres['PPD'] > 2) & (dres[1] > tr) ]
dres1 = dres1.sort_values('PNT', ascending=False)
print(data_path)
print("Дней:", tr)
#print(dres1)
pd.set_option('display.precision', 4)
#with pd.option_context('display.max_columns', 12):
print(dres1)


Data/BTC-XZC/thirtyMin/
Дней: 70
           0      1    2     3     4     5     6     PPD      PNT
476   0.3430  228.0  0.0   5.0  42.0  60.0   6.0  4.8999  78.2020
1891  0.3643  212.0  0.0   7.0  44.0  56.0   6.0  5.2045  77.2348
1216  0.3040  245.0  0.0   6.0  44.0  56.0   6.0  4.3434  74.4901
478   0.3210  226.0  0.0   5.0  42.0  60.0   8.0  4.5864  72.5564
479   0.3210  226.0  0.0   5.0  42.0  60.0   9.0  4.5864  72.5564
480   0.3210  226.0  0.0   5.0  42.0  60.0  10.0  4.5864  72.5564
1221  0.3293  219.0  0.0   6.0  44.0  58.0   6.0  4.7043  72.1172
1895  0.3413  210.0  0.0   7.0  44.0  56.0  10.0  4.8759  71.6759
1893  0.3413  210.0  0.0   7.0  44.0  56.0   8.0  4.8759  71.6759
1894  0.3413  210.0  0.0   7.0  44.0  56.0   9.0  4.8759  71.6759
2654  0.4413  159.0  0.0   8.0  46.0  60.0   9.0  6.3039  70.1622
1220  0.2840  243.0  0.0   6.0  44.0  56.0  10.0  4.0575  69.0182
1219  0.2840  243.0  0.0   6.0  44.0  56.0   9.0  4.0575  69.0182
1218  0.2840  243.0  0.0   6.0  44.0  56.0 

In [34]:
res = np.asarray(res)
dres = pd.DataFrame(res)

dres['ProfitPerDay'] = dres[0]/tr * 1000
#dres1 = dres[(dres[0] > 0.02) ]
dres1 = dres[(dres['ProfitPerDay'] > 2) & (dres[1] > 20) & (dres[2] == 15) & (dres[5] == 10)   ]
dres1 = dres1.sort_values('ProfitPerDay', ascending=False)
print(data_path)
print("Дней:", tr)
print(dres1)


Data/BTC-RISE/fiveMin/
Дней: 20
                0     1     2     3     4     5  ProfitPerDay
1955  0.073277758  42.0  15.0  43.0  70.0  10.0   3.663887907
2005  0.063413207  44.0  15.0  44.0  70.0  10.0   3.170660325
1905  0.063189193  40.0  15.0  42.0  70.0  10.0   3.159459650
1855  0.058441852  38.0  15.0  41.0  70.0  10.0   2.922092596
1975  0.057027156  25.0  15.0  43.0  74.0  10.0   2.851357793
1960  0.054088441  33.0  15.0  43.0  71.0  10.0   2.704422051
1965  0.051730568  32.0  15.0  43.0  72.0  10.0   2.586528391
1970  0.050178774  25.0  15.0  43.0  73.0  10.0   2.508938709
1980  0.049873470  23.0  15.0  43.0  75.0  10.0   2.493673502
1805  0.047724852  37.0  15.0  40.0  70.0  10.0   2.386242619
2010  0.047594643  36.0  15.0  44.0  71.0  10.0   2.379732129
2055  0.046979608  44.0  15.0  45.0  70.0  10.0   2.348980418
1910  0.044553060  32.0  15.0  42.0  71.0  10.0   2.227653005
2015  0.044032675  35.0  15.0  44.0  72.0  10.0   2.201633744
1985  0.043520852  21.0  15.0  43.0  7

In [80]:
#Длина торговли по времени
ss = df_trades['Date']
sst = ss.apply(lambda x:time.mktime(time.strptime(x, "%Y-%m-%dT%H:%M:%S")))
sst = sst.diff()/60
df_trades['TL'] = sst
sst[sst<3].count()

0

In [39]:
dres1.groupby(5).sum()

,0,1,2,3,4,ProfitPerDay
5,,,,,,
9.0,1.728198074,664.0,72.0,318.0,366.0,43.204951859
10.0,1.027989925,416.0,40.0,182.0,206.0,25.699748114
11.0,1.699812487,659.0,72.0,318.0,366.0,42.495312177
12.0,1.699812487,659.0,72.0,318.0,366.0,42.495312177


In [33]:
#Массовая обработка по одному временному интервалу

res = []

def fo(x):

    profit, num_trades = RSI03(x, df) 
    
    res.append([market_name, profit, num_trades] + x.tolist() + [volume])
    return profit

data_path = 'Data/*/fiveMin/'
time_interval = 30
day = 24/(time_interval/60) #Зависит от временного интервала
buyCorrection = 1.003
sellCorrection = 0.997

fn = glob(data_path)
for cur_dir in fn:
    cur_file = glob(cur_dir +'*.csv')[0]
    market_name = cur_file.split('\\')[1]
    print(market_name)
    df = pd.read_csv(cur_file);
    volume = df['BV'].mean() * day
    brute(fo, ((5,15,1), (28,48,2), (50,80,2), (6,11,1)), finish=None)
    
res30 = res    

BTC-ADX
BTC-BAT
BTC-BCC
BTC-DASH
BTC-ETC
BTC-ETH
BTC-KMD
BTC-LSK
BTC-LTC
BTC-MCO
BTC-NEO
BTC-OK
BTC-OMG
BTC-PAY
BTC-PIVX
BTC-POWR
BTC-QTUM
BTC-RISE
BTC-SALT
BTC-STRAT
BTC-WAVES
BTC-XEM
BTC-XMR
BTC-XRP
BTC-XVG
BTC-XZC
BTC-ZEC
USDT-BTC


In [1]:

res = np.asarray(res)
dres = pd.DataFrame(res)
dres.iloc[:,1:] = dres.iloc[:,1:].astype(float)

NameError: name 'np' is not defined

In [13]:
dres['ProfitPerDay'] = dres[1]/tr * 1000
xx = dres[(~dres[0].isin(['BTC-XVG', 'USDT-BTC'])) & (dres[1] > 0.01)]

xx1 = xx.groupby(0).max()
xx1.sort_values('ProfitPerDay', ascending=False)

,1,2,3,4,5,6,7,ProfitPerDay
0,,,,,,,,
BTC-PIVX,0.575356224,586.0,10.0,46.0,78.0,10.0,252.794677777,9.432069248
BTC-ADX,0.553845582,623.0,10.0,46.0,78.0,10.0,1039.003110251,9.079435776
BTC-XZC,0.397233545,289.0,10.0,46.0,78.0,10.0,316.004593160,6.512025335
BTC-RISE,0.362761987,344.0,10.0,46.0,78.0,10.0,213.715323962,5.946917817
BTC-MCO,0.330411328,610.0,10.0,46.0,78.0,10.0,1189.197165925,5.416579141
BTC-OMG,0.296256320,256.0,10.0,46.0,78.0,10.0,3377.736619122,4.856660979
BTC-BAT,0.291351559,306.0,10.0,46.0,78.0,10.0,467.118943213,4.776255072
BTC-LSK,0.146223642,242.0,10.0,40.0,78.0,10.0,1269.308490820,2.397108884
BTC-KMD,0.117248803,224.0,10.0,46.0,78.0,10.0,306.421132727,1.922111521


In [10]:
xx1.sort_values('ProfitPerDay', ascending=False)

,1,2,3,4,5,6,7,ProfitPerDay
0,,,,,,,,
BTC-RISE,0.280123082,191.0,10.0,46.0,78.0,10.0,244.450449102,4.592181674
BTC-XZC,0.187121991,179.0,10.0,46.0,78.0,10.0,361.333308491,3.067573623
BTC-BCC,0.158093716,114.0,10.0,46.0,78.0,10.0,16594.712289609,2.591700265
BTC-PIVX,0.132325668,167.0,10.0,46.0,78.0,10.0,218.106558600,2.169273245
BTC-SALT,0.123700778,130.0,10.0,46.0,74.0,10.0,291.982655781,2.027881600
BTC-OK,0.092035685,99.0,10.0,46.0,78.0,10.0,766.711641210,1.508781729
BTC-KMD,0.087329572,141.0,10.0,46.0,78.0,10.0,355.743701420,1.431632330
BTC-DASH,0.076051592,68.0,10.0,46.0,78.0,10.0,1263.472501007,1.246747407
BTC-ADX,0.070142157,121.0,10.0,46.0,76.0,10.0,383.949195272,1.149871431
